In [1]:
%load_ext autoreload
%autoreload 2

In [1]:
import sys

sys.path.insert(0, '../')
from src.mongo_utils import collection, create_set, get_text, get_database_entry
from src.utils import sanitize_context, sanitize_context_word
import regex as re

In [20]:
def get_all_entries():
    query = {"$and": [{"beispiele": {"$exists": True}}, {"bedeutungen": {"$exists": True}}]}

    # Retrieving documents that match the query
    return collection.find(query)

In [117]:
test = get_all_entries()
test = list(test)

In [118]:
def evaluate_sets(sets):
    def compare_words(words):
        # Check if all words in the list are the same
        if len(words) < 2:
            return True  # If there's only one word or none, they are considered "equal"
        first_word = words[0]
        return all(word == first_word for word in words)
    
    complain_data = []
    for title, context_word, context_sentence, gt in sets:
        complain = False
        if context_word and re.match("[^\p{L}]", context_word):
            context_word = sanitize_context_word(context_word)
            if title != context_word:
                complain = True
        extracted_context = re.findall(r"''(.*?)''", context_sentence)
        extracted_context = [sanitize_context(i) for i in extracted_context]
        a = []
        for x in extracted_context:
            if not re.match(r"\[\[(.*?)\]\]", x):
                a.append(x.strip())
        extracted_context = a
        if len(extracted_context) > 1 and context_word is not None:
            if not compare_words(extracted_context) and context_word != extracted_context[0]:# and title not in extracted_context:
                complain = True
        # if re.match("^(?!.*[\p{L}]).*$", gt):
        #     complain = True
            
        if complain:
            complain_data.append(
                [title, context_word, context_sentence, gt]
            )
        
    return complain_data

In [119]:
from pprint import pprint
import json

complaints = []
for a in test[:]:
    complaints += evaluate_sets(create_set(a))

In [120]:
len(complaints)

173

In [116]:
filter_list = ["anschlagen", "nicht", "Schlickerguss", "Förderband", "κληροδότης", "Grundzahlwort", "Blust", "Abänderungsplan", "âne", "ungültige", "5000", "Landpostbotin", "-atr"]

for a in complaints:
    if a[0] in filter_list:
        print(a)

['Akkadist', '1886', "(''1886'':) „Denn nicht nur ist unsere Kenntnis [des Sumerischen und des Akkadischen] sehr mangelhaft, was auch der gläubigste ''Akkadist'' oder Sumerist einräumen wird, sondern es erhellt mehr und mehr, daſs die Lesung mancher Wörter, deren man gewiſs zu sein glaubte, noch keineswegs feststeht, […].“", 'akademische Person, die sich mit akkadischer Sprache, Kultur und Geschichte befasst']
['hinkriegen', 'Kollege', "''[Kollege:]'' „Können Sie mir helfen, ich ''kriege'' das nicht ''hin.''“", "''etwas hinkriegen:''  eine Sache mit Können erledigen / schaffen"]
['Idealtyp', 'Ideal-)Typen', "„Skalička unternimmt den entscheidenden Schritt in Richtung moderne Sprachtypologie, indem er ''(Ideal-)Typen'', basierend auf vielen sprachlichen Merkmalen, konstruiert.“", 'Konstrukt eines Sprachtyps, bei dem alle interagierenden Eigenschaften optimal aufeinander abgestimmt sind']
['anschlagen', 'schlagen', "„Bestimmte Chemotherapien ''[schlagen]'' offenbar besser ''[an]'', wenn 

In [121]:
for a in complaints:
    print(f"'{a[0]}' --> {sanitize_context_word(a[1])}")

'Auto' --> Autos
'eg' --> *eg*
'g' --> *g*
'g' --> *g*
'dóttir' --> -dóttir
'hedonisch' --> hedonischen
'spongiform' --> spongiforme
'triggern' --> triggert
'5' --> 5
'AM' --> AM
'sic' --> sic
'NGO' --> NGO
'NGO' --> NGOs
'NGO' --> NGO
'AR' --> ar
'Status' --> -Status
'son' --> -sönerna
'de' --> De
'y' --> Y
'-prozentig' --> 100-prozentig
'Kommerzialist' --> Kommerzialisten
'2.' --> 2
'2.' --> 2
'1.' --> 1
'1.' --> 1
'3.' --> 3
'3.' --> 3
'4.' --> 4
'4.' --> 4
'5.' --> 5
'5.' --> 5
'6.' --> 6
'6.' --> 6
'7.' --> 7
'7.' --> 7
'8.' --> 8
'8.' --> 8
'9.' --> 9
'9.' --> 9
'10.' --> 10
'10.' --> 10
'Donga' --> Dongas
'Donga' --> Dongas
'Donga' --> Dongas
'Ikonodule' --> Ikonodulen
'aufpassen wie ein Heftelmacher' --> {{L|passte|paßte}}
'-ose' --> -ose
'phys.' --> phys
'kollaterieren' --> Kollaterieren
'Idealtypologie' --> Ideal-)Typologie
'avarice' --> avarice
'omniszient' --> omniszienten
'gesteuert' --> -gesteuerte
'Ismus' --> -Ismus
'Personalmaskulinum' --> Personalmaskulina
'Nisse' --> 

In [87]:
complaints[:10]

[['Auto',
  'Autos',
  "„Von Calderón sind ca. 120 Dramen (sog. Comedias), 80 Fronleichnamsspiele ''(Autos'' sacramentales) erhalten.“",
  'einaktiges religiöses oder später auch profanes Theaterstück in Spanien und Portugal, das an Festtagen öffentlich aufgeführt wurde'],
 ['eg',
  '*eg*',
  "now you get what you deserve ''*eg*''",
  "''Netzjargon:'' evil grin"],
 ['Konifere',
  'Taxus',
  "Die Eibe ''(Taxus)'' ist eine ''Konifere,'' obwohl sie keine Zapfen trägt.",
  "''Botanik:'' Nadelgehölz"],
 ['g',
  '*g*',
  "meinst du das wirklich ??? ''*g*''",
  "''schriftlicher Internetjargon:'' Abkürzung für ''grin'' (englisch) oder ''grins'' (deutsch), oft in Chats oder Foren im Internet verwendet; bringt zum Ausdruck, dass der Autor grinst, lächelt, oder einfach amüsiert ist"],
 ['g',
  '*g*',
  "Diese Abkürzung kann auch gesteigert werden, so wird aus ''*g*'' in manchen Fällen *gg* oder *ggg*, wenn der Autor sehr fröhlich ist.",
  "''schriftlicher Internetjargon:'' Abkürzung für ''grin'' 

In [25]:
print(json.dumps(complaints))

[["Auto", "(Autos", "\u201eVon Calder\u00f3n sind ca. 120 Dramen (sog. Comedias), 80 Fronleichnamsspiele ''(Autos'' sacramentales) erhalten.\u201c", "einaktiges religi\u00f6ses oder sp\u00e4ter auch profanes Theaterst\u00fcck in Spanien und Portugal, das an Festtagen \u00f6ffentlich aufgef\u00fchrt wurde"], ["1", "1.", "Take S train on track ''1.''\nNehmen Sie die S-Bahn auf Gleis ''1.''", "eine arabische Ziffer (ein Zahlzeichen), welche die Zahl Eins darstellt"], ["Rasen", "(Rasen)", "Einige Merkmale: L\u00e4ngsspur-Aufzeichnung, damit mechanisches Schneiden des Bandes m\u00f6glich; feststehende, mehrspurige K\u00f6pfe; Bandbreite 12,65 mm; Bandgeschwindigkeit 76,2 cm/s; Tonspurbreite je 0,17 mm; Abstand zwischen Tonspuren ''(Rasen)'' 50 \u00b5m k\u00fcrzeste Wellenl\u00e4nge = 2 bit = 1,98 \u00b5m Datenrate/Kanal 1,152 Mbit/ s; Aufzeichnungsdichte 1,52 kbit/mmz; Band auf Spule.", "Schutzzone in der Video- beziehungsweise Tontechnik"], ["Rasen", "(Rasen)", "Die Aufzeichnungsspurbreite

In [7]:
test = create_set(get_database_entry('Phylum'))

In [18]:
print(test[0][2])

Dem zoologischen ''Phylum'' (Stamm) entspricht in der Botanik formal die ''Diviso'' (Abteilung).


In [19]:
sanitize_context(test[0][2])

'Dem zoologischen Phylum (Stamm) entspricht in der Botanik formal die Diviso (Abteilung).'